In [1]:
import pandas as pd
import numpy as np

https://www.kaggle.com/datasets/jackywang529/toronto-bikeshare-data

# 1. Dataloading

In [3]:
BSD = pd.read_csv(r"C:\Users\William Hazen\OneDrive - University of Toronto\Term 2 (W)\MIE 1613\MIE 1613 Project\BikeShare Data\Bike share ridership 2022-08.csv")

In [4]:
BSD

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,17515458,1174,7259,08/01/2022 00:00,Lower Spadina Ave / Lake Shore Blvd,7712.0,08/01/2022 00:20,NaN,3328,Casual Member
1,17515440,468,7208,08/01/2022 00:00,80 Clinton St (North of College),7534.0,08/01/2022 00:07,Walnut Ave / Queen St W,4662,Casual Member
2,17515442,1217,7259,08/01/2022 00:00,Lower Spadina Ave / Lake Shore Blvd,7712.0,08/01/2022 00:20,NaN,4510,Casual Member
3,17515441,1124,7269,08/01/2022 00:00,Toronto Eaton Centre (Yonge St),7076.0,08/01/2022 00:18,York St / Queens Quay W,1013,Casual Member
4,17515443,1992,7052,08/01/2022 00:00,Wellington St W / Bay St,7193.0,08/01/2022 00:33,Queen St W / Gladstone Ave,4017,Casual Member
...,...,...,...,...,...,...,...,...,...,...
705145,18317445,1670,7100,08/31/2022 23:58,Dundas St E / Regent Park Blvd,7322.0,09/01/2022 00:26,King St W / Brant St,6417,Casual Member
705146,18317447,574,7245,08/31/2022 23:58,Tecumseth St / Queen St W - SMART,7069.0,09/01/2022 00:08,Queen St W / Spadina Ave,141,Annual Member
705147,18317448,533,7257,08/31/2022 23:59,Dundas St W / St. Patrick St,7037.0,09/01/2022 00:08,Bathurst St / Dundas St W,1583,Casual Member
705148,18317449,615,7368,08/31/2022 23:59,Lisgar St / Dundas St SMART,7521.0,09/01/2022 00:09,Emerson Ave / Bloor St W,2059,Casual Member


In [5]:
BSD.describe()

,Trip Id,Trip Duration,Start Station Id,End Station Id,Bike Id
count,7.051500e+05,7.051500e+05,705150.000000,705020.000000,705150.000000
mean,1.791772e+07,1.013313e+03,7279.587922,7277.692268,3959.081465
std,2.313256e+05,3.778077e+03,201.853670,201.316650,2119.053790
min,1.751544e+07,0.000000e+00,7000.000000,7000.000000,15.000000
25%,1.771830e+07,4.570000e+02,7100.000000,7100.000000,2209.000000
50%,1.791801e+07,7.600000e+02,7252.000000,7250.000000,4114.000000
75%,1.811841e+07,1.197000e+03,7429.000000,7427.000000,5821.000000
max,1.831745e+07,1.232765e+06,7722.000000,7722.000000,7263.000000
